In [200]:
import pdfplumber
import PyPDF2

from PIL import Image

import requests
import re


import pandas as pd

# 下載會考試題

In [201]:
"""
    https://cap.rcpet.edu.tw/exam/103/103P_Nature.pdf
    https://cap.rcpet.edu.tw/exam/104/104P_Nature.pdf
    https://cap.rcpet.edu.tw/exam/105/105P_Nature150DPI.pdf
    https://cap.rcpet.edu.tw/exam/106/106P_Nature150DPI.pdf
    https://cap.rcpet.edu.tw/exam/107/107P_Nature150DPI.pdf
    https://cap.rcpet.edu.tw/exam/108/108P_Nature150DPI.pdf
    https://cap.rcpet.edu.tw/exam/109/109P_Nature150DPI.pdf
    
    https://cap.rcpet.edu.tw/exam/103/103P_Chinese.pdf    
    https://cap.rcpet.edu.tw/exam/103/103P_English.pdf
    https://cap.rcpet.edu.tw/exam/103/103P_Math.pdf
    https://cap.rcpet.edu.tw/exam/103/103P_Society.pdf
    https://cap.rcpet.edu.tw/exam/103/103P_Nature.pdf        
    https://cap.rcpet.edu.tw/exam/103/103P_Writing.pdf
"""

subjects = ['Chinese',
            'English',
            'Math',
            'Society',
            'Nature']

url_1 = 'https://cap.rcpet.edu.tw/exam/'


url_lists = []        #會考檔案的網址
filename_lists = []   #會考檔案的網址

for year in range(103, 110):
    for subject in subjects:
        if year < 105 or year == 107 :
            url_DPI = ''
        else:
            url_DPI = '150DPI'

        new_url = url_1 + str(year) + '/' + str(year) + 'P_' + subject + url_DPI + '.pdf'
        print(new_url)        
        filename = str(year) + '_' + subject + '.pdf'
        
        url_lists.append(new_url)
        filename_lists.append(filename)
url_lists
            


https://cap.rcpet.edu.tw/exam/103/103P_Chinese.pdf
https://cap.rcpet.edu.tw/exam/103/103P_English.pdf
https://cap.rcpet.edu.tw/exam/103/103P_Math.pdf
https://cap.rcpet.edu.tw/exam/103/103P_Society.pdf
https://cap.rcpet.edu.tw/exam/103/103P_Nature.pdf
https://cap.rcpet.edu.tw/exam/104/104P_Chinese.pdf
https://cap.rcpet.edu.tw/exam/104/104P_English.pdf
https://cap.rcpet.edu.tw/exam/104/104P_Math.pdf
https://cap.rcpet.edu.tw/exam/104/104P_Society.pdf
https://cap.rcpet.edu.tw/exam/104/104P_Nature.pdf
https://cap.rcpet.edu.tw/exam/105/105P_Chinese150DPI.pdf
https://cap.rcpet.edu.tw/exam/105/105P_English150DPI.pdf
https://cap.rcpet.edu.tw/exam/105/105P_Math150DPI.pdf
https://cap.rcpet.edu.tw/exam/105/105P_Society150DPI.pdf
https://cap.rcpet.edu.tw/exam/105/105P_Nature150DPI.pdf
https://cap.rcpet.edu.tw/exam/106/106P_Chinese150DPI.pdf
https://cap.rcpet.edu.tw/exam/106/106P_English150DPI.pdf
https://cap.rcpet.edu.tw/exam/106/106P_Math150DPI.pdf
https://cap.rcpet.edu.tw/exam/106/106P_Society150

['https://cap.rcpet.edu.tw/exam/103/103P_Chinese.pdf',
 'https://cap.rcpet.edu.tw/exam/103/103P_English.pdf',
 'https://cap.rcpet.edu.tw/exam/103/103P_Math.pdf',
 'https://cap.rcpet.edu.tw/exam/103/103P_Society.pdf',
 'https://cap.rcpet.edu.tw/exam/103/103P_Nature.pdf',
 'https://cap.rcpet.edu.tw/exam/104/104P_Chinese.pdf',
 'https://cap.rcpet.edu.tw/exam/104/104P_English.pdf',
 'https://cap.rcpet.edu.tw/exam/104/104P_Math.pdf',
 'https://cap.rcpet.edu.tw/exam/104/104P_Society.pdf',
 'https://cap.rcpet.edu.tw/exam/104/104P_Nature.pdf',
 'https://cap.rcpet.edu.tw/exam/105/105P_Chinese150DPI.pdf',
 'https://cap.rcpet.edu.tw/exam/105/105P_English150DPI.pdf',
 'https://cap.rcpet.edu.tw/exam/105/105P_Math150DPI.pdf',
 'https://cap.rcpet.edu.tw/exam/105/105P_Society150DPI.pdf',
 'https://cap.rcpet.edu.tw/exam/105/105P_Nature150DPI.pdf',
 'https://cap.rcpet.edu.tw/exam/106/106P_Chinese150DPI.pdf',
 'https://cap.rcpet.edu.tw/exam/106/106P_English150DPI.pdf',
 'https://cap.rcpet.edu.tw/exam/106

In [ ]:
#下載pdf
#解析url的filename
i = 0
for url in url_lists:
    r = requests.get(url, allow_redirects=True)
    open('會考題目/' + filename_lists[i], 'wb').write(r.content)
    print(filename_lists[i])
    i += 1

In [19]:
pdffile ='會考題目/' + filename_lists[0]     #pdf檔路徑及檔名
pdf = pdfplumber.open(pdffile)
p0 = pdf.pages[3]
text = p0.extract_text()       #讀文字
print(pdffile)
print(text)

會考題目/103_Society.pdf
None


# 解析PDF

In [202]:
def extract_image(page):
    try:
        if '/XObject' in page['/Resources']:
            xObject = page['/Resources']['/XObject'].getObject()
            for obj in xObject:
                if xObject[obj]['/Subtype'] == '/Image':
                    size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
                    data = xObject[obj].getData()
                    if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                        mode = "RGB"
                    else:
                        mode = "P"
                    # 保存图片的文件名前缀
                    img_pre = str(int(time.time()*1000))
                    if '/Filter' in xObject[obj]:
                        if xObject[obj]['/Filter'] == '/FlateDecode':
                            img = Image.frombytes(mode, size, data)
                            img.save(img_pre + ".png")
                        elif xObject[obj]['/Filter'] == '/DCTDecode':
                            img = open(img_pre + ".jpg", "wb")
                            img.write(data)
                            img.close()
                        elif xObject[obj]['/Filter'] == '/JPXDecode':
                            img = open(img_pre + ".jp2", "wb")
                            img.write(data)
                            img.close()
                        elif xObject[obj]['/Filter'] == '/CCITTFaxDecode':
                            img = open(img_pre + ".tiff", "wb")
                            img.write(data)
                            img.close()
                    else:
                        img = Image.frombytes(mode, size, data)
                        img.save(img_pre + ".png")
        else:
            print("no image")
    except:
        print("image extract fail")

In [203]:
def extract_content(pdf_path):
    # 内容提取，使用 pdfplumber 打开 PDF，用于提取文本
    with pdfplumber.open(pdf_path) as pdf_file:
        # 使用 PyPDF2 打开 PDF 用于提取图片
        pdf_image_reader = PyPDF2.PdfFileReader(open(pdf_path, "rb"))
        print(pdf_image_reader.getNumPages())

        content = ''
        # len(pdf.pages)为PDF文档页数，一页页解析
        for i in range(len(pdf_file.pages)):
            #print("第 %s 頁" % i)
            # pdf.pages[i] 是读取PDF文档第i+1页
            page_text = pdf_file.pages[i]
            # page.extract_text()函数即读取文本内容
            page_content = page_text.extract_text()
            if page_content:
                content = content + page_content + "\n"
                #print(page_content)

            # 提取图片
            #page_image = pdf_image_reader.getPage(pageNumber=i)
            #extract_image(page_image)
    return content

In [9]:
#取得pdf內文
filename = filename_lists[18]
pdf_file ='會考題目/' + filename     #pdf檔路徑及檔名
print(filename)
text = extract_content(pdf_file)
text

106_Society.pdf
14


'請考生依指示\n填寫准考證末兩碼 \n年國中教育會考\n106\n社 會 科 試 題 本\n請不要翻到次頁！\n讀完本頁的說明，聽從監試委員的指示才開始作答！\n※ 請先確認你的答案卡、准考證與座位號碼是否一致無誤。\n請閱讀以下測驗作答說明：\n測驗說明：\n這是國中教育會考社會科試題本，試題本採雙面印刷，共 13 頁，有 \n63 題選擇題，每題都只有一個正確或最佳的答案。測驗時間從 08：30 \n到 09：40，共 70 分鐘。作答開始與結束請聽從監試委員的指示。\n注意事項：\n1. 所有試題均為四選一的選擇題，答錯不倒扣。\n2. 試題中所附圖形，如有附上比例尺，以比例尺為依據作答；若無 \n比例尺，則該圖僅作為參考，不代表實際大小。\n3. 可利用試題本中空白部分計算，切勿在答案卡上計算。\n4. 依試場規則規定，答案卡上不得書寫姓名座號，也不得作任何標記。\n故意汙損答案卡、損壞試題本，或在答案卡上顯示自己身分者，該科\n考試不予計列等級。\n作答方式：\n請依照題意從四個選項中選出一個正確或最佳的答案，並用 2B 鉛筆在答\n案卡上相應的位置畫記，請務必將選項塗黑、塗滿。如果需要修改答案，\n請使用橡皮擦擦拭乾淨，重新塗黑答案。例如答案為 B，則將     選項塗\n黑、塗滿，即：      \n以下為錯誤的畫記方式，可能導致電腦無法正確判讀。如：\n       —未將選項塗滿\n       —未將選項塗黑\n       —未擦拭乾淨\n       —塗出選項外\n       —同時塗兩個選項\n請聽到鐘（鈴）響起，於試題本右上角\n方格內填寫准考證末兩碼，再翻頁作答\n一、單題：(1～55題)\n1.  目前因婚姻關係來臺灣的新移民已超過50萬人，政府透過許多方式協助他們\n適應臺灣社會，例如建置數位資訊平臺，以多國語文的網頁介面，提供數位\n學習及文化交流的空間。上述多國語文最可能涵括下列何者？\n(A)俄羅斯文、德文\n(B)越南文、印尼文\n(C)阿拉伯文、土耳其文\n(D)西班牙文、葡萄牙文\n2.  圖(一)為臺灣某次馬拉松比賽路線圖，參賽者在路跑過\n程中，最可能看到下列何種景觀？\n(A)岬角與海灣相間\n(B)潟湖、沙洲遍布\n(C)大規模的隆起珊瑚礁\n(D)陡直的峭壁、深切的峽谷\n圖(一)\n3.  2014年，

In [197]:
# Matches if ... matches next, but doesn’t consume any of the string. This is called a lookahead assertion.
matches = re.findall(r'(?=\n(\d{1,2}\.{1}\s{2}.*?)\n\d{1,2}\.{1}\s{2})', text, re.DOTALL)

for match in matches:
    match = match.replace('\n', '').replace('\r', '').replace(' ', '').replace('　', '')
    print(match)


1.目前因婚姻關係來臺灣的新移民已超過50萬人，政府透過許多方式協助他們適應臺灣社會，例如建置數位資訊平臺，以多國語文的網頁介面，提供數位學習及文化交流的空間。上述多國語文最可能涵括下列何者？(A)俄羅斯文、德文(B)越南文、印尼文(C)阿拉伯文、土耳其文(D)西班牙文、葡萄牙文
2.圖(一)為臺灣某次馬拉松比賽路線圖，參賽者在路跑過程中，最可能看到下列何種景觀？(A)岬角與海灣相間(B)潟湖、沙洲遍布(C)大規模的隆起珊瑚礁(D)陡直的峭壁、深切的峽谷圖(一)
3.2014年，中國與甲國簽署了乳製品的合作協議，由於甲國向來以酪農業發達著稱，兩國合作將有助於提高中國乳製品的品質與產量。甲國最可能為下列何者？(A)北韓(B)荷蘭(C)泰國(D)挪威
4.圖(二)為臺灣某月分的月平均等雨量線圖，根據圖中雨量的分布狀況判斷，該月分臺灣的氣候特色為下列何者？(A)高溫多對流雨(B)易形成滯留鋒(C)東北季風盛行(D)易受颱風侵襲圖(二)
5.某國的國旗以白色為底，上有藍色十字，充分展現該國的自然與人文特色。因該國有四分之一的國土在北極圈內，氣候寒冷，故白色象徵冬季白雪覆蓋的國土；藍色象徵境內廣布的冰蝕湖、河流；十字表示該國在歷史上與其他北歐鄰國的密切關係。上文是在介紹何國的國旗？(A)丹麥(B)芬蘭(C)瑞士(D)俄羅斯1請翻頁繼續作答
6.下列為臺灣某份刊物上刊載的文章：今日針對普及衛生思想，監督指導的單位須以身作則，首先應喚起市街庄行政人員的自覺，有效配合地方上保甲的力量。此外，藉由小學校、公學校對衛生教育的提倡、社會上報章雜誌的宣傳，來提升民眾的衛生意識。根據內容判斷，此段文章主要在討論下列何者？(A)荷蘭聯合東印度公司推動衛生教育(B)中華民國政府規畫與施行衛生教育(C)清朝開港通商時，傳教士引進公共衛生觀念(D)日本統治時期，政府與民間推廣公共衛生觀念
7.圖(三)是某機構的性質及工作內容示意圖。據此判斷，該機構最可能為下列何者？(A)郊&郊＝行郊(B)公行(C)市舶司(D)總理各國事務衙門圖(三)
8.表(一)為清朝後期幾位地方總表(一)督、巡撫的出身資料，根據內姓名民族經歷曾任職務容判斷，他們得以出任總督、巡左宗棠1861年浙江巡撫撫，與下列何者關係最密切？皆為楊岳斌皆為1864年陜甘總督(A)鴉片戰爭湘軍劉坤一漢人1865年江西巡撫將領(B)英法聯軍曾國荃1

In [226]:
#跑迴圈將所有PDF的內文存入df
subjects = ['Chinese',
            'English',
            'Math',
            'Society',
            'Nature']



question_list = []

for year in range(104,110):
    for subject in subjects:
        #取得pdf內文
        filename = str(year) + '_' + subject

        pdf_file ='會考題目/' + filename + '.pdf'    #pdf檔路徑及檔名
        print(filename)
        text = extract_content(pdf_file)


        #matches = re.findall('\\n(\d{1,2}\.{1}\s{2}.*?？)', text, re.DOTALL)
        matches = re.findall(r'(?=\n(\d{1,2}\.{1}\s{2}.*?)\n\d{1,2}\.{1}\s{2})', text, re.DOTALL)
        print(matches)

        #取出題號
        question_num_list = []
        for question in matches:
            question_num = re.match('^(\d{1,2})', question, re.DOTALL)
            question_num_list.append(question_num[0])

        #將各題分開放在list            
        for i in range(len(matches)):
            if subject == 'English':
                match = matches[i].replace('\n', '').replace('\r', '').replace('　', '')
            else:
                match = matches[i].replace('\n', '').replace('\r', '').replace(' ', '').replace('　', '')
            question = [year, subject, question_num_list[i] , match]
            print(year, subject, question_num_list[i] , match)
            question_list.append(question)

df = pd.DataFrame(question_list , columns = ['年度', '科目','題號','題目'])


    

104_Chinese
14
['1.  小敬搬新家，阿嬌欲送禮金表達祝賀之意。下列題辭，何者最適合寫在賀儀封套上？\n(A)近悅遠來                   (B)造福桑梓                (C)宜室宜家                (D)喬遷之喜', '2.  下列文句，何者文意最為通暢？\n(A)因為你不去，我還是要去\n(B)他年紀既然小，膽子卻不小\n(C)雖然這是第一次，所以做得很好 \n(D)她的作品不是不好，而是缺少創意', '3.  「當部分亞洲國家為了生產足夠的稻米而焦頭爛額之際，泰國稻米的產量卻多到一\n年出口九百萬噸，是全世界最大稻米輸出國。泰國成為世界米倉的原因是：民營企\n業非常有效率，擁有大量平坦肥沃的三角洲和充沛的水源，再加上政府機關支\n持民間銀行貸款給小農購買種子、肥料。以上種種，使泰國在米糧危機中占盡\n利益。」這段文字的主旨為下列何者？\n(A)泰國米質提升的技術                          (B)泰國稻米盛產的原因\n(C)泰國稻米行銷的方式                          (D)泰國復興米業的計畫', '4.  下列詞語「」中的字音，何組相同？\n(A)風聲鶴「唳」／暴「戾」之氣\n(B)「揖」讓而升／查「緝」走私\n(C)「擣」米去糠／陶「鑄」品行\n(D)再三叮「嚀」／滿地泥「濘」', '5.  「時然後言，人不厭其言。」這句話的涵義，與下列何者最接近？\n(A)沉默是金，可免禍從口出                  (B)善傾聽者，聞言必加審查\n(C)適時發言，方為人所接受                  (D)經常練習，言語方能動聽', '6.  下列詞語，何者沒有錯別字？\n(A)營私舞幣                   (B)武功祕笈                (C)試目以待                (D)時代先軀', '7.  下列選項「」中的疊字，何者是用來描摹聲音的狀聲詞？\n(A)「嘖嘖」稱奇的老師                          (B)「侃侃」而談的官員\n(C)「滔滔」不絕的名嘴                          (D)「聲聲」叮嚀的母親', '8.  小王和小林第一次碰面，下列

14
['1.  Look at the picture.  The girl drew two            on the wall.\n(A) circles\n(B) lines\n(C) points\n(D) squares   ', '2.  This dress is pretty,            it does not look good on me.\n(A) so  (B) but  (C) or  (D) if', '3.  Getting up early on a cold morning is not easy,           ?\n(A) are you  (B) do you  (C) does it  (D) is it', '4.\t Tonight\tI’ll\tstay\tat\tthe\toffice\tuntil\tI\t           the work.\n(A)\tfinish\t (B)\tam\tfinishing\t (C)\tfinished\t (D)\twill\tfinish', '5.  Charles            a day in the department store looking for a hat for his wife.\n(A) cost  (B) spent  (C) saw  (D) made', '6.  Tom            ten pounds over the past two months.  He looks much thinner now.\n(A) loses  (B) has lost  (C) will lose  (D) was losing', '7.  I don’t like any one of these three watches.  Can you show me            one?\n(A) the others  (B) other  (C) either  (D) another ', '8.  The waiters are asked to be           ; they should always smile and remember to say \n“Welcom

['1.  算式 (−1 ) × (−3 ) ×   之值為何？\n2 4 3\n1\n(A) \n4\n11\n(B) \n12\n11\n(C) \n4\n13\n(D) \n4', '2.  已知直線 L 的方程式為 x = 3，直線 M 的方程式為 y = −2，判斷下列何者為\n直線 L、直線 M 畫在坐標平面上的圖形？\n(A)               (B)\n(C)                                      (D)', '3.  下列各選項中的盒狀圖分別呈現出某班四次小考數學成績的分布情形，\n       哪一個盒狀圖呈現的資料其四分位距最大？\n(A)                                      (B)\n(C)                                      (D)\n請翻頁繼續作答\n1\n6\n2', '4.  算式 (−3)4 − 72 −   之值為何？\n(−2)3\n(A) −138\n(B) −122 \n(C) 24\n(D) 40', '5.  如圖(一)，  AB 為圓 O 的直徑，  BC 為圓 O 的 C\n一弦， 自 O 點作 BC 的垂線， 且交 BC 於 D 點。\nD\n若 AB = 16，  BC = 12，則 △OBD 的面積為何？\n(A) 6 7 A B\nO\n(B) 12 7\n(C) 15\n(D) 30\n圖(一)', '6.  計算多項式 −2x (3x − 2)2 + 3除以 3x − 2 後，  所得商式與餘式兩者之和為何？\n(A) −2x + 3\n(B) −6x2 + 4x\n(C) −6x2 + 4x + 3\n(D) −6x2 − 4x + 3', '7.  將圖 ( 二 ) 的正四角錐 ABCDE 沿著其中的四個邊剪開後，  形成的展開圖為\n圖 ( 三 )。判斷下列哪一個選項中的四個邊可為此四個邊？\n(A) AC 、  AD 、  BC 、  DE\n(B) AB 、  BE 、  DE 、  CD\n(C) AC 、  BC 、  AE 、  DE\n(D) AC 、  AD 、  AE 、  BC\n圖(二) 圖(三)\n2', '8.  下列哪一個選項中的等式不成立？\n(A) 

14
['1.   圖(一)是生物進行某種生理作用的示意圖，圖中箭頭 (cid:31)(cid:30)(cid:29)(cid:28)\n代表能量或物質在葉片中的進出，此生理作用最可\n(cid:30)(cid:25)\n能是下列何者？\n(cid:23)(cid:22)\n(cid:27)(cid:26)\n(A)呼吸作用\n(B)蒸散作用\n(C)光合作用\n(D)觸發運動\n(cid:24)\n圖(一)', '2.   小臻取石墨、硫、鋁和銀四種物質中的其中一個，來進行如圖(二)所示的二個\n實驗，根據實驗結果判斷，她最可能是取哪一個物質來進行實驗？\n(A)石墨\n(B)硫\n(C)鋁\n(D)銀\n圖(二)', '3.   小樺突然想到海邊觀察星象，而去請教氣象專家。專家說現在天氣圖上本地\n是被標記為「L」的天氣系統籠罩，建議他改天再去。專家會做出如此建議，可\n能是當此天氣系統籠罩時，將會造成下列何種影響？\n(A)天空不利雲層發展，容易有月光造成光害\n(B)海水會是滿潮狀態，海邊可活動空間較少\n(C)大氣中水氣易凝結，觀星的視野會受遮蔽\n(D)紫外線指數會偏高，不適合安排戶外活動', '4.   某人分別於甲、乙二區種植具有抗蟲基因的棉花及一般棉花，中間以道路相隔，\n如圖(三)所示。經過一段時間後，發現乙區的棉花也具有此抗蟲基因，產生此現\n象的原因，最可能是棉花的下列哪一構造傳播所造成？\n(A)花柱                                           (B)花粉\n(C)胚珠                                           (D)子房\n圖(三)', '5.   若以箭頭方向表示電流方向，則下列選項中哪一個電路裝置表示的電流方向正\n確？\n(A)     (B)    (C)    (D) \n1\n請翻頁繼續作答', '6.   某地區樹林中棲息著一種蛾，依身體顏色可分成深色蛾和淺色蛾，其主要的天敵\n為鳥類。圖(四)為某段期間內深色蛾及淺色蛾數量變化的示意圖，圖中灰色部分標\n記為甲、乙兩時期。若依天擇說解釋甲或乙時期蛾的數量變化，下列何者最合理？\n(A)在甲時期內鳥類較易發現深色蛾\n(B)在甲時期內淺色蛾突變成深色蛾\n(C)在乙時期內鳥類的數量逐年下降\n(

['1.    「考期將至，望著桌上堆積如山的書本，我的心在瞬間彷彿變成了鉛塊。當下只\n有悔恨，悔恨自己未能善加運用時間。」作者在句中以「鉛塊」來比喻他當下\n的哪一種心情？\n(A)煩躁\u3000\u3000\u3000\u3000          (B)堅定\u3000\u3000\u3000\u3000         (C)麻木\u3000\u3000\u3000\u3000          (D)沉重', '2.    「荷馬史詩是永遠的傑作，但不能永遠以它為範本。」這句話的意旨與下列何\n者最接近？\n(A)敏銳的鑑賞力是創作優良作品的前提\n(B)創作必須求新求進，不能一味地模仿\n(C)再偉大的作品，價值也會隨時間而減損\n(D)作家過度被崇拜時，往往難以自我超越\n3.\n(cid:31)(cid:31)(cid:30)(cid:30)(cid:29)(cid:29)(cid:28)(cid:28)\n(cid:27)(cid:27)(cid:26)(cid:26)(cid:25)(cid:25)\n(cid:31)(cid:31)(cid:30)(cid:29)(cid:28) (cid:3)(cid:2)(cid:5)(cid:4)，(cid:127)(cid:129)\n(cid:27)(cid:26)(cid:25)(cid:24)(cid:23)(cid:22)(cid:21) (cid:141)(cid:28)(cid:143)(cid:144)(cid:16)(cid:157)\n(cid:20)(cid:19)(cid:18)(cid:17)(cid:16)(cid:15)(cid:14)(cid:13)(cid:12)(cid:11)(cid:10)   €‚ƒ„ƒ…ƒ†‡ƒ(cid:15)\n(cid:9)(cid:8)(cid:7)(cid:6)(cid:28)(cid:5)(cid:4)(cid:26) ˆˆ‰(cid:27)Š‹Œ\nŽ‘(cid:127)’“(cid:144)\n”•–—˜™\n關於這段詩句的解說，下列敘述何者正確？\n(A)本詩主旨在於藉由鳥類不同的生活型態，比喻人生窮通禍福的境遇\n(B)詩中運用人性化筆法，藉鳥之口控訴人類的貪婪與對待禽鳥的惡行\n(C)詩中的「我」以無私的心態來對待眾鳥，不

...

['1.    \n      \n     \n根據這張圖表，下列何者的症狀最可能是中暑？\n(A)冒汗而體溫正常的小健\n(B)體溫過高不出汗的小康\n(C)大量流汗虛弱頭暈的小平\n(D)心跳加速體溫正常的小安', '2.    「寧可讓學得快的人等，也不能讓不會的人繼續不會。」下列何者最符合這句話\n所強調的教育理念？\n(A)強化資優教育，培養菁英                  \n(B)提倡忍讓，加強品格教育\n(C)視學生資質，給予適當教導              \n(D)無論良與莠，一個都不放棄', '3.    \n人生何嘗不像一場競賽。跑得慢的人有些會怨天尤人，一路上窩囊委屈；\n有的卻抱著龜兔賽跑的精神，努力不懈直到終點。而那些跑得快的也不見得永\n遠領先，有的後繼無力被人超前，有的半路摔倒一蹶不起，選擇放棄。因此，\n________________________ 。\n根據文意，畫線處填入下列何者最恰當？\n(A)人生的競賽往往是暗潮洶湧的 \n(B)臺前風光是背後汗水累積而成\n(C)勝敗皆由天定，無須怨天尤人\n(D)堅持到底的人才是最後的贏家 ', '4.   「野生動物隨著環境會發展出新的生活方式。城市中的灰色松鼠比鄉下的較快\n對威脅產生反應，牠們不吱吱叫驅趕敵人，而是垂下尾巴來警示敵人。」根\n據這段文字，下列敘述何者最不恰當？\n(A)城市松鼠會因環境改變毛色              \n(B)城市松鼠會以垂下尾巴示警\n(C)鄉下松鼠對威脅的反應較慢\n(D)鄉下松鼠以吱吱叫驅趕敵人                                                                               \n1\n請翻頁繼續作答', '5.        關於這則貼文與留言，下列解讀何者最恰當？\n (A)小畢認為善惡終有報\n (B)阿文的想法傾向樂觀\n (C)凱哥同意小畢的觀點\n (D)小安的看法近於小畢', '6.        下列文句，何者詞語使用最恰當？\n(A)酒後不開車，未免發生事故               \n(B)見到故居破敗，不免有些傷感\n(C)他毫無商量餘地，避免不近人情      \n(D)他決心改正遲到毛病，難免受處罰', '7.   「宋朝

14
['1.  Look at the picture.  The man is holding one of the _____. \n(A) bowls\n(B) glasses\n(C) plates\n(D) spoons', '2.  My mom puts the _____ on to keep her hands away from water when she washes\nthe dishes.\n(A) gloves      (B) jacket       (C) ring     (D) socks', '3.\t Can\tyou\t_____\tyour\tfeet\ta\tlittle\toff\tthe\tfloor?\t\tI\twant\tto\tsee\tif\tmy\tkeys\tare\tunder\tthe\t\nsofa.\n(A) check  (B) kick  (C) raise   (D) show\n   ', '4.\t _____\tof\tmy\tsisters\tare\tolder\tthan\tI\tam.\t\tI’m\tthe\tyoungest\tof\tthe\tthree\tchildren\tin\tmy\t\nfamily.    \n(A) Both  (B) Few  (C) Most  (D) Some', '5.  The beach near our town is a very _____ place.  Every summer at least one kid dies \nthere in the water. \n(A) dangerous   (B) exciting   (C) lonely   (D) popular ', '6.\t My\twife\tenjoys\tgoing\tto\tthe\tmovies\twith\ther\tfriends,\tbut\tI\tlike\tto\twatch\tTV\tat\thome\t\nby _____.\n(A)\tI\t (B)\tme\t (C)\tmyself\t (D)\tmine', '7.\t For\tthe\tpast\ttwenty\tyears,\tmy\tfather\t__

['1.  已知 a = ( −12 ) × ( −23 ) × ( −34 ) × ( −45 )，b = ( −123 ) × ( −234 ) × ( −345 )，\n判斷下列敘述何者正確？\n(A) a、b 皆為正數\n(B) a、  b 皆為負數\n(C) a 為正數，  b 為負數\n(D) a 為負數，  b 為正數', '2.  算式 23 × 53  之值為何？\n(A) 30\n(B) 90\n(C) 1000\n(D) 1000000', '3.  小真煮好了 25 顆湯圓，其中 15 顆為芝麻湯圓，10 顆為花生湯圓。已知小真\n想從煮好的湯圓中撈一顆，若每顆湯圓被小真撈到的機會相等，則他撈到花生\n湯圓的機率為何？\n1\n(A) \n2\n2\n(B) \n3\n2\n(C) \n5\n1\n(D) \n10', '4.  算式  2 × (  48 −  12 ) 之值為何？\n(A) 6 2\n(B) 2 6\n(C) 2 21\n(D) 4 6 − 2 3\n1\n請翻頁繼續作答', '5.  如圖 ( 一 )，平行四邊形 ABCD 中， ∠ A = 100°。\n若 ∠ABD：   ∠DBC = 3：   2，  則 ∠DBC 的度數為何？\n(A) 32\n(B) 40\n(C) 48\n(D) 60 圖(一)', '6.  圖 ( 二 ) 數線上的 A、  B、  C 三點所表示的數分別為 a、  b、  c，且原點為 O。\n根據圖中各點位置，判斷下列四個式子的值何者最大？\n(A)  a  +  b \n(B)  a  +  c \n圖(二)\n(C)  a − c \n(D)  b − c ', '7.  計算 2 x 2 − 3 除以 x + 1 後，得商式和餘式分別為何？\n(A) 商式為 2，餘式為 −5\n(B) 商式為 2 x − 5，餘式為 5\n(C) 商式為 2 x + 2，餘式為 −1\n(D) 商式為 2 x − 2，餘式為 −1', '8.  下列何者可表示成兩個質數的乘積？\n(A) 81\n(B) 82\n(C) 83\n(D) 84\n2', '9.  已知小薇住家的西方 100 公尺處為車站，住家的北方 200 公尺處為學校，且\n從學校往東方走 100 公尺，  再往南方走 400 公尺

['1.  小嵐到某國旅遊時，寄明信片回\n臺灣，圖(一)是明信片上具有該\n國特色的郵戳。根據郵戳上的資\n訊判斷，小嵐最可能到下列何國\n旅遊？\n(A)日本  (B)芬蘭\n(C)英國  (D)紐西蘭 圖(一)', '2.  根據中國氣象部門的統計，西北和 華北地區沙塵暴發生年均次數，1987至\n1999年期間為5.85次，2011至2015年期間則減少為3.5次。下列何者與上述環\n境問題的改善有密切關連？\n(A)河西走廊的防護林工程 \n(B)洞庭湖的退田還湖政策\n(C)三江平原的排水改善計畫 \n(D)藏南縱谷的退牧還林政策 \uf026 藏南縱谷＝藏南谷地', '3.  每年10月起至翌年4月底，捕撈鰻魚魚苗是臺灣沿海漁民傳統產業活動之一，\n更是許多漁民維持生計的重要經濟來源。農委會於2013年訂定相關捕撈規定，\n其中一項是：每年自3月1日至10月31日止，禁止於距岸三浬內海域、潮間帶及\n河口水域以任何方式捕撈鰻苗。上述規定最可能是為了達成下列何種目的？\n(A)推廣休閒漁業    (B)增加漁業資源\n(C)改變食用魚的種類  (D)避免國際漁事糾紛', '4.  政府計畫興建一條快速道路，規畫的路線如\n圖(二)所示。道路沿線有山地及河流阻隔，因\n此經過河谷需興建橋梁，遇到山脊則需開挖\n隧道，以求最短距離。根據圖(二)的等高線判\n斷，甲、乙、丙三個路段需興建的項目，依序\n為下列何者？\n(A)橋梁-隧道-橋梁 \n(B)隧道-橋梁-隧道\n圖(二)\n(C)隧道-隧道-橋梁 \n(D)橋梁-橋梁-隧道 N', '5.  圖(三)為位在臺灣北部的某工業區及其鄰近社區的 甲 乙\n分布圖，工業區排放廢氣會影響鄰近社區的空氣品\n工工業業區區\n質。若僅根據臺灣不同季節的盛行風向判斷，冬季\n時圖中哪一社區受該工業區廢氣的影響最大？ 丁 丙\n(A)甲  (B)乙\n(C)丙  (D)丁 圖(三)\n1', '6.  圖(四)為臺灣部分原住民族群的祭典與主\n矮\n要分布圖。若按圖片內容判斷，雅美族\n甲 靈\n(達悟族)應為圖中何者？ 祭 豐\n年\n(A)甲 丁\n祭\n(B)乙\n(C)丙\n飛\n(D)丁 猴\n乙 魚\n祭 丙\n祭\n圖(四)', '7.  圖(五)呈現某事件前夕，當地的社會\n情況。此事件最可能是下列何者

['1.  牛背鷺為一種鳥類，常飛至牛的背上，靠食用牛身上的寄生蟲與草地中的昆蟲\n維生。根據上述提及生物的交互關係，下列推論何者最合理？\n(A)牛背鷺與牛為共生關係\n(B)牛背鷺與寄生蟲為共生關係\n(C)寄生蟲主要寄生於牛背鷺與牛身上\n(D)寄生蟲與牛背鷺競爭牛身上的食物', '2.  如圖(一)所示，美美想把燒杯中的液體倒入滴定管中，她\n搭配下列哪一項器材來使用，最適合且最能避免在傾倒\n液體時灑出？\n(A)                  (B)                    (C)                   (D)\n圖(一)', '3.  澱粉在人體內經某種生理作用後可產生多個小分子 X，如圖(二)所示。有關此\n生理作用及小分子 X 的名稱，下列何者最合理？\n(A)消化作用，葡萄糖\n(B)消化作用，胺基酸\n(C)呼吸作用，葡萄糖 \n(D)呼吸作用，胺基酸 圖(二)', '4.  小喬找到有關「小白鷺、中白鷺、大白鷺」的資料，並整理如表(一)所示。根據\n此表，成年的小白鷺學名應為下列何者？\n表(一)\n(A) Ardea alba\n(B) Egretta alba\n(C) Egretta garzetta\n(D) Egretta intermedia', '5.  圖(三)為植物體內物質運輸示意圖，圖(四)為葉片氣孔狀態示意圖。白天植物進\n行旺盛的蒸散作用時，有關體內水分運輸方向(甲或乙)及葉片氣孔狀態(丙或丁)，\n下列組合何者正確？\n(A)甲，丙\n(B)甲，丁\n(C)乙，丙\n(D)乙，丁\n圖(三) 圖(四)\n1', '6.  小芳閉上眼睛做某種動作，其過程如圖(五)所示。當她從\n甲動作轉換成乙動作時，協調肌肉活動以維持平衡主要是\n由下列何者調控？\n(A)腦垂腺     \n(B)小腦\n(C)腦幹     \n(D)脊髓\n圖(五)', '7.  圖(六)為某區域的地層剖面示意圖，圖中相同符號或顏色的標示，代表其未受斷層錯\n動前為同一地層。已知此地區地層未曾發生上下翻轉，在地層乙中含有一千萬年前\n的生物化石，地層戊中則含有五千萬年前的生物化石。若圖中某地層含有三千萬年前\n的生物化石，則該地層最有可能為下列何者？\n(A)己或庚\n(B)丁或己\n(C)丙或丁\n(D)甲或丙\n

In [245]:
df.columns = ['年度', '科目', '題號', '題目']

In [246]:
df[df['題目'].str.contains('生物')]

,年度,科目,題號,題目
172,104,Nature,1,1.圖(一)是生物進行某種生理作用的示意圖，圖中箭頭(cid:31)(cid:30)(cid...
214,104,Nature,43,43.圖(二十四)為自然界中氮循環部分途徑的示意圖，其中甲、乙為能進行氮的轉變反應之生物。依...
402,105,Nature,7,7.已知某種具有葉綠體的原生生物會分解養分產生能量，推測該生物能否進行光合作用或呼吸作用，下...
411,105,Nature,16,16.表(二)為小慧列出家燕及家雨燕表(二)的分類資料，她推論「家燕和家雨燕在分類上為不同科...
680,107,Chinese,9,9.「自然界中，在透明度較高的水清水域，通常硝酸鹽、磷酸鹽等營養鹽的含量較少，相反地水濁時含...
794,107,Society,11,11.曾被飼養的流浪動物對人類較無戒心，但此特點卻被某些人利用，他們不但虐待流浪貓、狗，還將...
886,107,Nature,41,41.圖(三十一)為某兩條食物鏈依生物各階層所含能量的關係繪製成甲、乙能量塔之示意圖(面積不...
899,108,Chinese,1,1.「『饕餮』是一種古代怪獸，在生物圖鑑與動物園裡找不到，牠和蛟龍、鳳凰一樣，都是虛構的動物...
1010,108,Society,1,1.1854年某英國博物學家前往東印度群島考察，八年內總共蒐集了12萬餘件的生物標本，並著有...
1070,108,Society,61,61.關於文中提及企業主與農夫合作的方式，若雙方在合作內容簽訂生效後，對於內容發生爭議，以下...


In [248]:
df['題目'].iloc[1111]

'40.圖(十九)為某生態系中氮循環的部分過程，甲、乙分別代表微生物吸收、釋出含氮物質的作用，丙、丁代表在生物間轉換的含氮物質，關於甲～丁的推論，下列何者最合理？(A)甲：呼吸作用(B)乙：光合作用(C)丙：葡萄糖(D)丁：蛋白質圖(十九)'

In [249]:
df.to_csv('從pdf直存df.csv', index=False,encoding="utf_8_sig")